# Biomedical Image Segmentation Pipeline

## File structure setup

First we are going to setup our filestructure in order to use the Image IO Interface.  
What we want for the Image_interface is something like that:

```
data/
     imgname001/imaging.png
                segmentation.png
     imgname002/imaging.png
                segmentation.png
     imgname003/imaging.png
                segmentation.png
     ...
```

## File structure preparation

In [ ]:
# Import some libraries
import os
import glob
import shutil
from PIL import Image
import PIL.ImageOps
import numpy as np

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
print(f'GPU device availability: {physical_devices}')

# Configure data path for celltracking data set Fluo-N2DL-HeLa and file structure
path_common = "{define_your_path_to_images_here}"
path_ds_1 = os.path.join(path_common, "Fluo-N2DL-HeLa")
path_ds_clean = os.path.join(path_common, "clean_data")

img_type, img_format, seg_path = "grayscale", "tif", "_ST"
image_formats = {
    'tif': 'TIFF',
    'jpg': 'JPEG'
}

In [ ]:
# Initialize file structure
if not os.path.exists(path_ds_clean): os.mkdir(path_ds_clean)

# Iterate over both data sets
for ds in ["01", "02"]:
    # Define image directories
    path_ds_img = os.path.join(path_ds_1, ds)
    path_ds_seg = os.path.join(path_ds_1, ds + seg_path, "SEG")
    # Obtain sample list
    sample_list = os.listdir(path_ds_seg)
    # Remove every file which does not match image type and preprocess sample names
    for i in reversed(range(0, len(sample_list))):
        if not sample_list[i].endswith(".tif"):
            del sample_list[i]
        else:
            sample_list[i] = sample_list[i][7:]
    # Iterate over each sample and transform the data into desired file structure
    for sample in sample_list:
        index = 'SAMPLE_' + ds + '_' + sample[:-4]
        # Create sample directory
        path_sampleDir = os.path.join(path_ds_clean, index)
        if not os.path.exists(path_sampleDir): os.mkdir(path_sampleDir)
        # Copy image file into filestructure
        path_ds_sample_img = os.path.join(path_ds_img, "t" + sample)
        path_fs_sample_img = os.path.join(path_sampleDir, f"imaging.{img_format}")
        shutil.copy(path_ds_sample_img, path_fs_sample_img)  
        # Copy segmentation file into filestructure
        seg_file = "man_seg" + sample
        path_ds_sample_seg = os.path.join(path_ds_seg, seg_file)
        path_fs_sample_seg = os.path.join(path_sampleDir, f"segmentation.{img_format}")
        # Load segmentation from file
        seg_raw = Image.open(path_ds_sample_seg)
        # Convert segmentation from Pillow image to numpy matrix
        seg_pil = seg_raw.convert("LA")
        seg = np.array(seg_pil)
        # Keep only intensity and remove maximum intensitiy range
        seg_data = seg[:,:,0]
        # Union all separate cell classes to a single one
        seg_data[seg_data > 0] = 1
        # Transform numpy array back to a Pillow image & save to disk
        seg = Image.fromarray(seg_data)
        seg.save(path_fs_sample_seg, format=image_formats[img_format])

### Temporary fix for problems in [MIScnn](https://github.com/frankkramer-lab/MIScnn/pull/91)

In [ ]:
from miscnn.data_loading.interfaces import Image_interface

class Fixed_Image_interface(Image_interface):
    #---------------------------------------------#
    #                  load_image                 #
    #---------------------------------------------#
    def load_image(self, index):
        # Make sure that the image file exists in the data set directory
        img_path = os.path.join(self.data_directory, index)
        if not os.path.exists(img_path):
            raise ValueError(
                "Sample could not be found \"{}\"".format(img_path)
            )
        # Load image from file
        img_raw = Image.open(os.path.join(img_path, "imaging" + "." + \
                                          self.img_format))
        # Convert image to rgb or grayscale
        if self.img_type == "grayscale" and len(img_raw.getbands()) > 1 :
            img_pil = img_raw.convert("LA")
        elif self.img_type == "rgb" and img_raw.mode != "RGB":
            img_pil = img_raw.convert("RGB")
        else:
            img_pil = img_raw
        
        # Convert Pillow image to numpy matrix
        img = np.array(img_pil)
        # Keep only intensity for grayscale images
        if self.img_type == "grayscale" and len(img.shape) > 2:
            img = img[:, :, 0]
        # Return image
        return img, {"type": "image"}

### Image sample display

In [ ]:
# Import some libraries
from miscnn import Data_IO
from miscnn.data_loading.interfaces import Image_interface

# Initialize Data IO & Image Interface
interface = Fixed_Image_interface(classes=2, img_type=img_type, img_format=img_format)
data_io = Data_IO(interface, path_ds_clean, delete_batchDir=True)

# Obtain the sample list
sample_list = data_io.get_indiceslist()
sample_list.sort()

from IPython.display import display

# Load the first sample via MIScnn data loader
sample_test = data_io.sample_loader(sample_list[0], load_seg=True)

# Visualize the image
img_data = sample_test.img_data
print("Shape of image:", img_data.shape)
img = Image.fromarray(img_data[:,:,-1])
display(img)

# Visualize the ground truth segmentation
seg_data = sample_test.seg_data * 100
print("Shape of segmentation:", seg_data.shape)
seg = Image.fromarray(seg_data[:,:,-1])
display(seg)

## Define a custom architecture
### Multi-Path U-Net

In [ ]:
import tensorflow_addons as tfa
from tensorflow.keras.activations import relu
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, SpatialDropout2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Conv3DTranspose
from tensorflow.keras.layers import Reshape, Dropout, LeakyReLU, Attention, LayerNormalization
# Internal libraries/scripts
from miscnn.neural_network.architecture.abstract_architecture import Abstract_Architecture

#-----------------------------------------------------#
#           Architecture class: U-Net Plain           #
#-----------------------------------------------------#
""" The Plain variant of the popular U-Net architecture.
Methods:
    __init__                Object creation function
    create_model_2D:        Creating the 2D U-Net plain model using Keras
    create_model_3D:        Creating the 3D U-Net plain model using Keras
"""
class UNetMultiPath(Abstract_Architecture):
    #---------------------------------------------#
    #                Initialization               #
    #---------------------------------------------#
    def __init__(self):
        # Create list of filters
        self.feature_maps = {
            4: [40, 240],
            2: [40, 80, 160, 220]
        }

    #---------------------------------------------#
    #               Create 2D Model               #
    #---------------------------------------------#
    def create_model_2D(self, input_shape, n_labels=2):
        # Input layer
        inputs = Input(input_shape)
        # Cache contracting normalized conv layers
        # for later copy & concatenate links
        contracting_convs = {c: [] for c in self.feature_maps.keys()}
        all_middle_chains = []
        all_chains = []
        
        for stride, feature_map in self.feature_maps.items():
            # Start the CNN Model chain with adding the inputs as first tensor
            cnn_chain = inputs

            # Contracting layers
            for i in range(0, len(feature_map)):
                neurons = feature_map[i]
                cnn_chain = conv_layer_2D(cnn_chain, neurons, strides=1)
                cnn_chain = conv_layer_2D(cnn_chain, neurons, strides=1)
                cnn_chain = SpatialDropout2D(0.5)(cnn_chain)
                contracting_convs[stride].append(cnn_chain)
                cnn_chain = MaxPooling2D(pool_size=(stride, stride))(cnn_chain)

            # Middle Layer
            neurons = feature_map[-1]
            cnn_chain = conv_layer_2D(cnn_chain, neurons, strides=1, norm=False)
            cnn_chain = conv_layer_2D(cnn_chain, neurons, strides=1, norm=False)
            all_middle_chains.append(cnn_chain)
          
        cnn_chain1 = concatenate(all_middle_chains)
        cnn_chain1 = LayerNormalization(epsilon=1e-5)(cnn_chain1)
        
        for stride, feature_map in self.feature_maps.items():
            # Start the CNN Model chain with adding the inputs as first tensor
            cnn_chain = cnn_chain1
            # Expanding Layers
            for i in reversed(range(0, len(feature_map))):
                neurons = feature_map[i]
                cnn_chain = Conv2DTranspose(neurons, (stride, stride), 
                                            strides=(stride, stride),
                                            padding='same')(cnn_chain)
                cnn_chain = SpatialDropout2D(0.5)(cnn_chain)
                cnn_chain = concatenate([cnn_chain, contracting_convs[stride][i]], axis=-1)
                cnn_chain = conv_layer_2D(cnn_chain, neurons, strides=1)
                cnn_chain = conv_layer_2D(cnn_chain, neurons, strides=1)
            
            all_chains.append(cnn_chain)

        # Output Layer
        conv_out = Conv2D(n_labels, (1, 1), activation='softmax')(concatenate(all_chains, axis=-1))
        # Create Model with associated input and output layers
        model = Model(inputs=[inputs], outputs=[conv_out])
        # Return model
        return model

    #---------------------------------------------#
    #               Create 3D Model               #
    #---------------------------------------------#
    def create_model_3D(self, input_shape, n_labels=2):
        pass


#-----------------------------------------------------#
#                   Subroutines 2D                    #
#-----------------------------------------------------#
# Convolution layer
def conv_layer_2D(input, neurons, strides=1, norm=True):
    conv = Conv2D(neurons, (3, 3), padding='same', strides=strides)(input)
    if norm:
        conv = tfa.layers.InstanceNormalization(epsilon=1e-5)(conv)

    return LeakyReLU(alpha=0.1)(conv)

### U-Net++

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Average, concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Conv3DTranspose
from tensorflow.keras.regularizers import l2

# Internal libraries/scripts
from miscnn.neural_network.architecture.abstract_architecture import Abstract_Architecture

#-----------------------------------------------------#
#           Architecture class: U-Net++               #
#-----------------------------------------------------#
""" The Plain variant of the popular U-Net architecture.
Methods:
    __init__                Object creation function
    create_model_2D:        Creating the 2D U-Net plain model using Keras
    create_model_3D:        Creating the 3D U-Net plain model using Keras
"""
class UNetPlusPlus(Abstract_Architecture):
   
    def __init__(self):
        pass
    
    #---------------------------------------------#
    #               Create 2D Model               #
    #---------------------------------------------#
    def create_model_2D(self, input_shape, n_labels=2):
        nb_filter = [38,76,152,256,512]
        bn_axis = -1
        
        # Input layer
        img_input = Input(input_shape)
        
        conv1_1 = standard_unit(img_input, stage='11', nb_filter=nb_filter[0])
        pool1 = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(conv1_1)

        conv2_1 = standard_unit(pool1, stage='21', nb_filter=nb_filter[1])
        pool2 = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(conv2_1)

        up1_2 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up12', padding='same')(conv2_1)
        conv1_2 = concatenate([up1_2, conv1_1], name='merge12', axis=bn_axis)
        conv1_2 = standard_unit(conv1_2, stage='12', nb_filter=nb_filter[0])

        conv3_1 = standard_unit(pool2, stage='31', nb_filter=nb_filter[2])
        pool3 = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(conv3_1)

        up2_2 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up22', padding='same')(conv3_1)
        conv2_2 = concatenate([up2_2, conv2_1], name='merge22', axis=bn_axis)
        conv2_2 = standard_unit(conv2_2, stage='22', nb_filter=nb_filter[1])

        up1_3 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up13', padding='same')(conv2_2)
        conv1_3 = concatenate([up1_3, conv1_1, conv1_2], name='merge13', axis=bn_axis)
        conv1_3 = standard_unit(conv1_3, stage='13', nb_filter=nb_filter[0])

        conv4_1 = standard_unit(pool3, stage='41', nb_filter=nb_filter[3])
        pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(conv4_1)

        up3_2 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up32', padding='same')(conv4_1)
        conv3_2 = concatenate([up3_2, conv3_1], name='merge32', axis=bn_axis)
        conv3_2 = standard_unit(conv3_2, stage='32', nb_filter=nb_filter[2])

        up2_3 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up23', padding='same')(conv3_2)
        conv2_3 = concatenate([up2_3, conv2_1, conv2_2], name='merge23', axis=bn_axis)
        conv2_3 = standard_unit(conv2_3, stage='23', nb_filter=nb_filter[1])

        up1_4 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up14', padding='same')(conv2_3)
        conv1_4 = concatenate([up1_4, conv1_1, conv1_2, conv1_3], name='merge14', axis=bn_axis)
        conv1_4 = standard_unit(conv1_4, stage='14', nb_filter=nb_filter[0])

        conv5_1 = standard_unit(pool4, stage='51', nb_filter=nb_filter[4])

        up4_2 = Conv2DTranspose(nb_filter[3], (2, 2), strides=(2, 2), name='up42', padding='same')(conv5_1)
        conv4_2 = concatenate([up4_2, conv4_1], name='merge42', axis=bn_axis)
        conv4_2 = standard_unit(conv4_2, stage='42', nb_filter=nb_filter[3])

        up3_3 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up33', padding='same')(conv4_2)
        conv3_3 = concatenate([up3_3, conv3_1, conv3_2], name='merge33', axis=bn_axis)
        conv3_3 = standard_unit(conv3_3, stage='33', nb_filter=nb_filter[2])

        up2_4 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up24', padding='same')(conv3_3)
        conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], name='merge24', axis=bn_axis)
        conv2_4 = standard_unit(conv2_4, stage='24', nb_filter=nb_filter[1])

        up1_5 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up15', padding='same')(conv2_4)
        conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], name='merge15', axis=bn_axis)
        conv1_5 = standard_unit(conv1_5, stage='15', nb_filter=nb_filter[0])

        nestnet_output_1 = Conv2D(n_labels, (1, 1), activation='sigmoid', name='output_1', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_2)
        nestnet_output_2 = Conv2D(n_labels, (1, 1), activation='sigmoid', name='output_2', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_3)
        nestnet_output_3 = Conv2D(n_labels, (1, 1), activation='sigmoid', name='output_3', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_4)
        nestnet_output_4 = Conv2D(n_labels, (1, 1), activation='sigmoid', name='output_4', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_5)
        nestnet_output = Average()([nestnet_output_1, nestnet_output_2, nestnet_output_3, nestnet_output_4])

        return Model(inputs=[img_input], outputs=[nestnet_output])

    #---------------------------------------------#
    #               Create 3D Model               #
    #---------------------------------------------#
    def create_model_3D(self, input_shape, n_labels=2):
        pass
    

########################################
# 2D Standard
########################################

def standard_unit(input_tensor, stage, nb_filter, kernel_size=3):
    dropout_rate, act = 0.5, "relu"
    x = Conv2D(nb_filter, (kernel_size, kernel_size), activation=act, name='conv'+stage+'_1', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(input_tensor)
    x = Dropout(dropout_rate, name='dp'+stage+'_1')(x)
    x = Conv2D(nb_filter, (kernel_size, kernel_size), activation=act, name='conv'+stage+'_2', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(dropout_rate, name='dp'+stage+'_2')(x)

    return x

### MultiRes U-Net

In [ ]:
from tensorflow.keras.layers import Input, concatenate, BatchNormalization, Activation, add
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Conv3DTranspose
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import ELU, LeakyReLU
# Internal libraries/scripts
from miscnn.neural_network.architecture.abstract_architecture import Abstract_Architecture


#-----------------------------------------------------#
#         Architecture class: U-Net MultiRes          #
#-----------------------------------------------------#
""" The MultiRes variant of the popular U-Net architecture.

Methods:
    __init__                Object creation function
    create_model_2D:        Creating the 2D U-Net standard model using Keras
"""
class UNetMultiRes(Abstract_Architecture):
    #---------------------------------------------#
    #                Initialization               #
    #---------------------------------------------#
    def __init__(self, activation='sigmoid'):
        # Parse parameter
        self.activation = activation

    #---------------------------------------------#
    #               Create 2D Model               #
    #---------------------------------------------#
    def create_model_2D(self, input_shape, n_labels=2):
        # Input layer
        inputs = Input(input_shape)
        nb_filter = [38,76,152,300,600]

        mresblock1 = MultiResBlock_2D(nb_filter[0], inputs)
        pool1 = MaxPooling2D(pool_size=(2, 2))(mresblock1)
        mresblock1 = ResPath_2D(nb_filter[0], 4, mresblock1)

        mresblock2 = MultiResBlock_2D(nb_filter[1], pool1)
        pool2 = MaxPooling2D(pool_size=(2, 2))(mresblock2)
        mresblock2 = ResPath_2D(nb_filter[1], 3, mresblock2)

        mresblock3 = MultiResBlock_2D(nb_filter[2], pool2)
        pool3 = MaxPooling2D(pool_size=(2, 2))(mresblock3)
        mresblock3 = ResPath_2D(nb_filter[2], 2, mresblock3)

        mresblock4 = MultiResBlock_2D(nb_filter[3], pool3)
        pool4 = MaxPooling2D(pool_size=(2, 2))(mresblock4)
        mresblock4 = ResPath_2D(nb_filter[3], 1, mresblock4)

        mresblock5 = MultiResBlock_2D(nb_filter[4], pool4)

        up6 = concatenate([Conv2DTranspose(
            nb_filter[3], (2, 2), strides=(2, 2), padding='same')(mresblock5), mresblock4], axis=3)
        mresblock6 = MultiResBlock_2D(nb_filter[3], up6)

        up7 = concatenate([Conv2DTranspose(
            nb_filter[2], (2, 2), strides=(2, 2), padding='same')(mresblock6), mresblock3], axis=3)
        mresblock7 = MultiResBlock_2D(nb_filter[2], up7)

        up8 = concatenate([Conv2DTranspose(
            nb_filter[1], (2, 2), strides=(2, 2), padding='same')(mresblock7), mresblock2], axis=3)
        mresblock8 = MultiResBlock_2D(nb_filter[1], up8)

        up9 = concatenate([Conv2DTranspose(nb_filter[0], (2, 2), strides=(
            2, 2), padding='same')(mresblock8), mresblock1], axis=3)
        mresblock9 = MultiResBlock_2D(nb_filter[0], up9)

        conv10 = conv2d_bn(mresblock9, n_labels, 1, 1, activation=self.activation)

        model = Model(inputs=[inputs], outputs=[conv10])
        return model


    #---------------------------------------------#
    #               Create 3D Model               #
    #---------------------------------------------#
    def create_model_3D(self, input_shape, n_labels=2):
        pass

#-----------------------------------------------------#
#             Subroutines for 2D version              #
#-----------------------------------------------------#
def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1), activation='relu', name=None):
    '''
    2D Convolutional layers

    Arguments:
        x {keras layer} -- input layer
        filters {int} -- number of filters
        num_row {int} -- number of rows in filters
        num_col {int} -- number of columns in filters

    Keyword Arguments:
        padding {str} -- mode of padding (default: {'same'})
        strides {tuple} -- stride of convolution operation (default: {(1, 1)})
        activation {str} -- activation function (default: {'relu'})
        name {str} -- name of the layer (default: {None})

    Returns:
        [keras layer] -- [output layer]
    '''

    x = Conv2D(filters, (num_row, num_col), strides=strides, padding=padding, use_bias=False)(x)
    x = BatchNormalization(axis=3, scale=False)(x)

    if(activation == None):
        return x

    x = Activation(activation, name=name)(x)

    return x


def trans_conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(2, 2), name=None):
    '''
    2D Transposed Convolutional layers

    Arguments:
        x {keras layer} -- input layer
        filters {int} -- number of filters
        num_row {int} -- number of rows in filters
        num_col {int} -- number of columns in filters

    Keyword Arguments:
        padding {str} -- mode of padding (default: {'same'})
        strides {tuple} -- stride of convolution operation (default: {(2, 2)})
        name {str} -- name of the layer (default: {None})

    Returns:
        [keras layer] -- [output layer]
    '''

    x = Conv2DTranspose(filters, (num_row, num_col), strides=strides, padding=padding)(x)
    x = BatchNormalization(axis=3, scale=False)(x)

    return x


def MultiResBlock_2D(U, inp, alpha = 1.67):
    '''
    MultiRes Block

    Arguments:
        U {int} -- Number of filters in a corrsponding UNet stage
        inp {keras layer} -- input layer

    Returns:
        [keras layer] -- [output layer]
    '''

    W = alpha * U

    shortcut = inp

    shortcut = conv2d_bn(shortcut, int(W*0.167) + int(W*0.333) +
                         int(W*0.5), 1, 1, activation=None, padding='same')

    conv3x3 = conv2d_bn(inp, int(W*0.167), 3, 3,
                        activation='relu', padding='same')

    conv5x5 = conv2d_bn(conv3x3, int(W*0.333), 3, 3,
                        activation='relu', padding='same')

    conv7x7 = conv2d_bn(conv5x5, int(W*0.5), 3, 3,
                        activation='relu', padding='same')

    out = concatenate([conv3x3, conv5x5, conv7x7], axis=3)
    out = BatchNormalization(axis=3)(out)

    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = BatchNormalization(axis=3)(out)

    return out


def ResPath_2D(filters, length, inp):
    '''
    ResPath

    Arguments:
        filters {int} -- [description]
        length {int} -- length of ResPath
        inp {keras layer} -- input layer

    Returns:
        [keras layer] -- [output layer]
    '''


    shortcut = inp
    shortcut = conv2d_bn(shortcut, filters, 1, 1,
                         activation=None, padding='same')

    out = conv2d_bn(inp, filters, 3, 3, activation='relu', padding='same')

    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = BatchNormalization(axis=3)(out)

    for i in range(length-1):

        shortcut = out
        shortcut = conv2d_bn(shortcut, filters, 1, 1,
                             activation=None, padding='same')

        out = conv2d_bn(out, filters, 3, 3, activation='relu', padding='same')

        out = add([shortcut, out])
        out = Activation('relu')(out)
        out = BatchNormalization(axis=3)(out)

    return out


## MIScnn pipeline

Now, we can start setup our MIScnn pipeline.  
In this scenario, we just want to perform a training and cross-validation process.

### Define all preprocessing blocks & training hyperparams

In [ ]:
#### Import some libraries
import tensorflow as tf
from miscnn import Data_IO, Preprocessor, Data_Augmentation, Neural_Network
from miscnn.data_loading.interfaces import Image_interface
from miscnn.neural_network.metrics import tversky_crossentropy, dice_soft, \
                                          dice_crossentropy, tversky_loss
from miscnn.processing.subfunctions import Resize, Normalization

# Initialize Data IO & Image Interface
interface = Fixed_Image_interface(classes=2, img_type=img_type, img_format=img_format)
data_io = Data_IO(interface, path_ds_clean, delete_batchDir=True)

# Obtain the sample list
sample_list = data_io.get_indiceslist()
sample_list.sort()

# Create a pixel value normalization Subfunction for z-score scaling
sf_norm = Normalization(mode="z-score")
# Create a resizing Subfunction to shape 512x512
sf_resize = Resize((512, 512))
# Assemble Subfunction classes into a list
sf = [sf_resize, sf_norm]

epochs = 200
batch_size = 2
initial_learning_rate = 1e-4
decay = initial_learning_rate / epochs

pp = Preprocessor(data_io, batch_size=batch_size, subfunctions=sf,
                  prepare_subfunctions=True, prepare_batches=False,
                  data_aug=None, analysis="fullimage")

### Define auxiliary Dice-Similarity coefficient function  

In [ ]:
def calc_DSC(truth, pred, classes):
    dice_scores = []
    # Iterate over each class
    for i in range(classes):
        try:
            gt = np.equal(truth, i)
            pd = np.equal(pred, i)
            # Calculate Dice
            dice = 2*np.logical_and(pd, gt).sum() / (pd.sum() + gt.sum())
            dice_scores.append(dice)
        except ZeroDivisionError:
            dice_scores.append(0.0)
    # Return computed Dice Similarity Coefficients
    return dice_scores

### Start MIScnn pipeline with the complete CV code

In [ ]:
import re

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, accuracy_score
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from miscnn.neural_network.architecture.unet.plain import Architecture

def scheduler(epoch, lr):
    return lr * 1 / (1 + decay * epoch)

all_samples = np.array(sample_list)
kfold_splitter = KFold(n_splits=5, random_state=123, shuffle=True)
dsc_scores = []

# Create the Neural Network model
model = Neural_Network(preprocessor=pp, loss=tversky_crossentropy,
                       metrics=[dice_soft, dice_crossentropy],
                       batch_queue_size=10, learninig_rate=initial_learning_rate, 
                       architecture=UNetMultiRes())
print(model.model.summary())

for train_index, test_index in kfold_splitter.split(all_samples):
    model.reset_weights()
    # Define Callbacks
    cb_lr = LearningRateScheduler(scheduler, verbose=1)
    cb_mc = ModelCheckpoint('cfu_cv.model', monitor='val_dice_soft', save_best_only=True, verbose=3, mode='max')
    cb_es = EarlyStopping(monitor='loss', mode='min', min_delta=0.0001, patience=20, verbose=3)
    
    train_samples, test_samples = all_samples[train_index].tolist(), all_samples[test_index].tolist()
    model.evaluate(train_samples, test_samples, epochs=epochs, callbacks=[cb_lr, cb_es, cb_mc])
    
    model.load('cfu_cv.model', custom_objects={
              'dice_soft': dice_soft,
              'dice_crossentropy': dice_crossentropy,
              'tversky_crossentropy': tversky_crossentropy
          })
    model.predict(test_samples)
    
    for sample_test in test_samples:
        counts, visited = dict(), set()
        cfu_count = sample_test.split('_')[1].lower()

        sample_test0 = data_io.sample_loader(sample_test, load_seg=True, load_pred=True)
        dsc_scores.append(calc_DSC(sample_test0.seg_data, sample_test0.pred_data, classes=2))
        
    print(f'Running DSC (CFU): {np.mean([d[1] for d in dsc_scores])}±{np.std([d[1] for d in dsc_scores])}')
    print(f'Running DSC (all): {np.mean(dsc_scores)}±{np.std(dsc_scores)}')